### import tools

In [1]:
#from library import package
import pandas as pd
import numpy as np
import copy
import csv

#from file import function
from countrynamefix import countrynamefix
from rcp_ssp import clean_rcpssp_co2
from gas_coal import clean_gas_coal
from tong import clean_tong
from steel import process_steel
from create_consid import mk_consid_commit
from aggregation import aggregate
from expectations import make_expectation_over_data
from plotting import plot_data

### data cleaning and aggregation

In [2]:
## Matching country names across all data files
tong_cntry_fix, scen_cntry_fix, coal_cntry_fix, gas_cntry_fix, steel_cntry_fix = countrynamefix()

## Data cleaning
scen_CO2_cntry = clean_rcpssp_co2()
coal_panel, coalCO2_panel, gas_panel = clean_gas_coal(coal_cntry_fix, gas_cntry_fix)
steel_panel = process_steel()
tong_country_sector_CO2 = clean_tong(tong_cntry_fix)

# format consid/commit emissions by type
gas_consid, coal, steel_proposed, electricity_committed, otherindustry_proposed = mk_consid_commit()

# get all the data, and fix formatting
cntry_rgn_data = aggregate()
cntry_rgn_data = cntry_rgn_data.drop(columns = ['Unnamed: 0'])
drop_cntry = cntry_rgn_data.drop(columns = 'country')
rgn_data = drop_cntry.groupby(by = ['SSP', 'RCP', 'region']).sum().reset_index()
world_totals = rgn_data.drop(columns = 'region').groupby(by = ['SSP', 'RCP']).sum().reset_index()

# aggregate the data by region and country respectively, forming 2 expectations
expectation_rgn = make_expectation_over_data(cntry_rgn_data, 'region')
expectation_cntry = make_expectation_over_data(cntry_rgn_data, 'country')
expectation_world = make_expectation_over_data(cntry_rgn_data, ['SSP', 'RCP']).drop(columns = ['credence', 'posterior', 'p'])

names without matches written to a csv
manually corrected list of country names parsed
initial data inputs written to files with corrected country names


### plotting

In [ ]:
# create plots of emissions trajectories in each of the corresponding folders

# # plot the raw country and region level data
plot_data(rgn_data, 'raw', 'Planned, Committed, and Considered Emissions till 2100 ', 'data/plots/rgn_emissions_rcp_ssp_plots/')
plot_data(cntry_rgn_data, 'raw', 'Planned, Committed, and Considered Emissions till 2100 ', 'data/plots/cntry_emissions_rcp_ssp_plots/')

# plot the data aggregated over the entire world, still raw
plot_data(world_totals, 'raw', 'Planned, Committed, and Considered Emissions till 2100 ', 'data/plots/world_ssp_rcp_emissions/')

# plot the expectation across countries and regions, respectively
plot_data(expectation_rgn, 'raw', 'Total Expectable, Committed, and Considered Emissions till 2100 ', 'data/plots/rgn_exp_plots/')
plot_data(expectation_cntry, 'raw', 'Total Expectable, Committed, and Considered Emissions till 2100 ', 'data/plots/cntry_exp_plots/')